#  Compare Dataframes by key using pandas

Compare dataframes by a key:

1. "normalize both dataframe to same shape to compare with the df compare function
      - merge dataframe key columns together and indicate where they came from
      - concat unique rows from one data frame to the other so the number of rows is the same
      - sort based on key
1. use data frame compare function to compare rows


In [ ]:
import pandas as pd
import itertools
from pathlib import Path

In [1]:
argo_path = 'out/csv'
uscore_path = '/Users/ehaas/Documents/FHIR/US-Core/output/'
out_path_csv = 'out/compare_csv' 
out_path_excel = 'out/compare_excel'
#image_path = '/Users/ehaas/Documents/FHIR/US-Core/input/images'
image_path = 'out/image'
my_key = 'Path_id'

### clean up directory first 

In [ ]:
path = Path() / out_path_csv
path_list =  path.glob("*.csv")
[f.unlink() for f in path_list if f.is_file()]
path = Path() / out_path_excel
path_list =  path.glob("*.xlsx")
[f.unlink() for f in path_list if f.is_file()]

### Manual Create Profile Map

In [ ]:
file_map = {
'StructureDefinition-argo-device':'StructureDefinition-us-core-implantable-device',
'StructureDefinition-argo-immunization':'StructureDefinition-us-core-immunization',
#'StructureDefinition-argo-medicationstatement':'StructureDefinition-us-core-medicationstatement',
'StructureDefinition-argo-ethnicity':'StructureDefinition-us-core-ethnicity',
'StructureDefinition-argo-medication':'StructureDefinition-us-core-medication',
'StructureDefinition-argo-careteam':'StructureDefinition-us-core-careteam',
'StructureDefinition-argo-medicationorder':'StructureDefinition-us-core-medicationrequest',
'StructureDefinition-argo-vitalsigns':'StructureDefinition-us-core-vital-signs',
'StructureDefinition-argo-goal':'StructureDefinition-us-core-goal',
'StructureDefinition-argo-birthsex':'StructureDefinition-us-core-birthsex',
#'StructureDefinition-argo-profile-link':'StructureDefinition-us-core-profile-link',
'StructureDefinition-argo-procedure':'StructureDefinition-us-core-procedure',
'StructureDefinition-argo-observationresults':'StructureDefinition-us-core-observation-lab',
'StructureDefinition-argo-smokingstatus':'StructureDefinition-us-core-smokingstatus',
'StructureDefinition-argo-diagnosticreport':'StructureDefinition-us-core-diagnosticreport-lab',
'StructureDefinition-argo-condition':'StructureDefinition-us-core-condition-problems-health-concerns',
#'StructureDefinition-argo-resource-documentation':'StructureDefinition-us-core-resource-documentation',
'StructureDefinition-argo-documentreference':'StructureDefinition-us-core-documentreference',
'StructureDefinition-argo-race':'StructureDefinition-us-core-race',
'StructureDefinition-argo-patient':'StructureDefinition-us-core-patient',
# 'StructureDefinition-argo-careplan':'StructureDefinition-us-core-careplan',
}   

# df1 = pd.read_csv(r'test_files/StructureDefinition-C4BB-Coverage.csv')
# df2 = pd.read_csv(r'test_files/StructureDefinition-hrex-coverage.csv')

### Normalize Both Dataframe To Same Shape To Compare With The Df Compare Function

- save as csv and excel in out folders

In [ ]:
for k,v in file_map.items():
    print()
    print('='*80)
    print('='*80)
    print(f'comparing {k} with {v}')
    df1 = pd.read_csv(Path() / argo_path / f'{k}.csv' )
    df1 = df1.loc[:, ~df1.columns.str.contains("Mapping")] #drop mapping columns
    df1 = df1.applymap(lambda x: x.strip() if isinstance(x, str) else x) #trim
    df1['Path_id'] = df1['Path'] # create unique Path_id
    for i, slice in enumerate(df1['Slice Name']):
        # print(i, df1.Path_id[i], df1['Slice Name'][i] )
        if slice==slice:  # not a NaN
           df1['Path_id'][i]=f"{df1.at[i,'Path_id']}-{df1.at[i,'Slice Name']}"  # ignore warning for now
    #     print(i, df1.Path_id[i], df1['Slice Name'][i] )
    #     print()
    # with pd.option_context("display.max_rows", 1000):
    #    display(df1.Path_id)


    df2 = pd.read_csv(Path() / uscore_path / f'{v}.csv' )
    df2 = df2.loc[:, ~df2.columns.str.contains("Mapping")] #drop mapping columns
    df2 = df2.applymap(lambda x: x.strip() if isinstance(x, str) else x) #trim
    df2['Path_id'] = df2['Path'] # create unique Path_id
    for i, slice in enumerate(df2['Slice Name']):
        # print(i, df2.Path_id[i], df2['Slice Name'][i] )
        if slice==slice:  # not a NaN
           df2['Path_id'][i]=f"{df2.at[i,'Path_id']}-{df2.at[i,'Slice Name']}" # ignore warning for now
    #     print(i, df2.Path_id[i], df2['Slice Name'][i] )
    #     print()
    # with pd.option_context("display.max_rows", 1000):
    #     display(df2.Path_id)
    # break

    print(f'df1.shape={df1.shape}, df2.shape={df2.shape}')
    

    
    df_all = df1[[my_key]].merge(df2[[my_key]], on=my_key, 
                       how='outer', indicator=True)

    both = df_all.query("_merge == 'both'")
    left_only = df_all.query('_merge == "left_only"')
    right_only = df_all.query('_merge == "right_only"')

    #print(f'df_all=\n{df_all}')
    print(f'both={both.shape}')
    print(f'Argo DQ only={left_only.shape}')
    print(f'US Core only={right_only.shape}')

    df1_new = pd.concat([df1,right_only], ignore_index=True)
    df1_new = df1_new.set_index(my_key)
    df1_new = df1_new.sort_index()

    #print(df1_new.head())

    df2_new = pd.concat([df2,left_only], ignore_index=True)
    df2_new = df2_new.set_index(my_key)
    df2_new = df2_new.sort_index()

    #print(df2_new.head())
    print(f'df1_new.shape={df1_new.shape}, df2_new.shape={df2_new.shape}')
    
    try:
        df_compare = df1_new.compare(df2_new, align_axis = 1)
    except ValueError as e:
        print('='*80)
        print(e)
        print('='*80)
        print('argo','us-core')
        
        print(df1_new.columns == df2_new.columns)
        print('df2_new', df2_new.shape)
        
        for i, my_index in enumerate(df2_new.index):
            try:
             print(i, my_index, df1_new.index[i])
            except:
                print('is this the problem?', i,my_index)
    print("k", k, type(k))
    print("v", v, type(v))

    #df_compare.head()
    print(f"printing {out_path_excel}/compare-{k.replace('StructureDefinition-','')}-{v.replace('StructureDefinition-','')}.xlsx...")
    df_compare.to_excel(f"{out_path_excel}/compare-{k.replace('StructureDefinition-','')}-{v.replace('StructureDefinition-','')}.xlsx")
    print(f"printing {out_path_csv}/compare-{k.replace('StructureDefinition-','')}-{v.replace('StructureDefinition-','')}.csv...")
    df_compare.to_csv(f"{out_path_csv}/compare-{k.replace('StructureDefinition-','')}-{v.replace('StructureDefinition-','')}.csv")

### Compress Output Folders to Zip files 

- includes a static README.txt file explaining how to read

In [3]:
%%bash -s "$out_path_excel" "$out_path_csv" "$image_path"
echo "================================================================="
echo "===zip up csv and excel files and put in==="
echo "===$3/argo-compare.zips file for downloads==="
echo "================================================================="
zip -j $3/argo-compare-excel.zip $1/*.*
zip -j $3/argo-compare-csv.zip $2/*.*

===zip up csv and excel files and put in===
===out/image/argo-compare.zips file for downloads===
updating: compare-argo-birthsex-us-core-birthsex.xlsx (deflated 8%)
updating: compare-argo-careteam-us-core-careteam.xlsx (deflated 5%)
updating: compare-argo-condition-us-core-condition-problems-health-concerns.xlsx (deflated 6%)
updating: compare-argo-device-us-core-implantable-device.xlsx (deflated 7%)
updating: compare-argo-diagnosticreport-us-core-diagnosticreport-lab.xlsx (deflated 5%)
updating: compare-argo-documentreference-us-core-documentreference.xlsx (deflated 8%)
updating: compare-argo-ethnicity-us-core-ethnicity.xlsx (deflated 8%)
updating: compare-argo-goal-us-core-goal.xlsx (deflated 5%)
updating: compare-argo-immunization-us-core-immunization.xlsx (deflated 8%)
updating: compare-argo-medication-us-core-medication.xlsx (deflated 7%)
updating: compare-argo-medicationorder-us-core-medicationrequest.xlsx (deflated 4%)
updating: compare-argo-observationresults-us-core-observatio